In [2]:
import pandas as pd
import numpy as np
from unidecode import unidecode

# Ajustar la cantidad de caracteres que se muestran en una celda
pd.set_option('display.max_colwidth', None)



In [3]:
# Cargar archivo CSV desde el sistema local
ruta_archivo = "./noticias.csv"
df = pd.read_csv(ruta_archivo,low_memory=False)

In [4]:
df.head(5)

,titulo,bajada,fecha,cuerpo,fuente,url,fecha_scraping,cuerpo_limpio
0,Militar chileno es herido tras enfrentamiento con migrantes en Colchane,El incidente se registró en la frontera de Chile con Bolivia. Un segundo uniformado acudió al rescate de su compañero mientras estaba siendo atacado.,2024-08-11,"Un militar resultó heridos tras un violento enfrentamiento con migrantes, que serían de nacionalidad boliviana, en las cercanías de Colchane. Durante el incidente, los atacantes sustrajeron el fusil de cargo de uno de los militares, el cual estaba sin cargadores, y lo golpearon en la cabeza. \nEl hecho tuvo lugar el pasado sábado a las 19:30 horas aproximadamente, en uno de los puestos de observación fronterizo entre Chile y Bolivia. Todo habría comenzado en el momento en que el militar intentó impedir el paso al grupo de migrantes al país, de acuerdo a lo consignado por Emol. \n\nEn medio de este ataque, un segundo militar acudió al rescate de su compañero, justo en el momento en que este estaba siendo atacado, logrando que detuvieran los golpes. Tras esto, ambos fueron trasladados al Hospital de Iquique, centro asistencial del cual el militar que fue golpeado con su fusil fue dado de alta este domingo. \nPor otro lado, su compañero habría resultado con un esguince en su tobillo. \n \nAnte este ataque, el alcalde de Colchane, Javier García, se pronunció al respecto condenando lo sucedido. ""Es inaceptable que migrantes irregulares ataquen de forma violenta al personal del Ejército de nuestras fronteras"", inició comentando el edil. \nPosteriormente agregó que es ""urgente que el Gobierno y el Congreso Nacional trabajen para elaborar leyes que permitan efectivamente impedir el ingreso al país de migrantes irregulares. Asímismo se debe de dotar de mayor facultad al Ejército para usar la fuerza en caso de ser necesario, pues es la única forma de entregar seguridad a nuestros vecinos que viven en la frontera. Finalmente quiero hacer un llamado al Gobierno de Bolivia a colaborar y poner mayor atención de lo que sucede en la frontera, ya que este país debe dejar de ser el centro operativo continental del tráfico de migrantes y también del crimen organizado"", finalizó García. \nPersonal de Carabineros y personal del Ejército se desplegaron en la zona para poder encontrar el arma que usaron para atacar al militar, así como también a los protagonistas del incidente. Tras esto, pudo ser encontrado el fusil en las cercanías de los hechos. \nA raíz de estos ataques es que se interpuso una denuncia a Carabineros por los delitos de maltrato de obra a personal de las Fuerzas Armadas y robo de material de guerra. Además, se iniciará una investigación sumaria administrativa, para determinar eventuales responsabilidades. \n\n",24horas,https://www.24horas.cl/actualidad/nacional/militar-es-herido-tras-enfrentamiento-con-migrantes-en-colchane,2024-08-12 09:39:25.413305,militar resultó heridos tras violento enfrentamiento migrantes nacionalidad boliviana cercanías colchane incidente atacantes sustrajeron fusil cargo militares cargadores golpearon cabeza \n hecho lugar pasado sábado 1930 horas aproximadamente puestos observación fronterizo chile bolivia comenzado momento militar intentó impedir paso grupo migrantes país acuerdo consignado emol \n\n medio ataque segundo militar acudió rescate compañero justo momento atacado logrando detuvieran golpes tras ambos trasladados hospital iquique centro asistencial militar golpeado fusil dado alta domingo \n lado compañero resultado esguince tobillo \n \n ataque alcalde colchane javier garcía pronunció respecto condenando sucedido inaceptable migrantes irregulares ataquen forma violenta personal ejército fronteras inició comentando edil \nposteriormente agregó urgente gobierno congreso nacional trabajen elaborar leyes permitan efectivamente impedir ingreso país migrantes irregulares asímismo debe dotar mayor facultad ejército usar fuerza caso ser necesario pues única forma entregar seguridad vecinos viven f

In [5]:
df.columns

Index(['titulo', 'bajada', 'fecha', 'cuerpo', 'fuente', 'url',
       'fecha_scraping', 'cuerpo_limpio'],
      dtype='object')

In [6]:
df.shape

(315057, 8)

In [7]:
# Buscar datos nulos en cada columna
nulos_por_columna = df.isnull().sum()

# Mostrar el número de nulos por columna
print(nulos_por_columna)

# También puedes ver si hay nulos en todo el DataFrame
nulos_totales = df.isnull().sum().sum()
print(f'Total de valores nulos en el DataFrame: {nulos_totales}')

titulo            55958
bajada            56685
fecha                 0
cuerpo                0
fuente                0
url                   0
fecha_scraping        0
cuerpo_limpio         0
dtype: int64
Total de valores nulos en el DataFrame: 112643


In [8]:
# Mostrar las primeras 5 filas de la columna 'cuerpo_limpio'
print(df['cuerpo_limpio'].head())

0     militar resultó heridos tras  violento enfrentamiento  migrantes    nacionalidad boliviana   cercanías  colchane   incidente  atacantes sustrajeron  fusil  cargo     militares     cargadores   golpearon   cabeza \n hecho  lugar  pasado sábado   1930 horas aproximadamente     puestos  observación fronterizo  chile  bolivia   comenzado   momento    militar intentó impedir  paso  grupo  migrantes  país  acuerdo   consignado  emol \n\n medio   ataque  segundo militar acudió  rescate   compañero justo   momento      atacado logrando  detuvieran  golpes tras  ambos  trasladados  hospital  iquique centro asistencial    militar   golpeado   fusil  dado  alta  domingo \n  lado  compañero  resultado   esguince   tobillo \n \n  ataque  alcalde  colchane javier garcía  pronunció  respecto condenando  sucedido  inaceptable  migrantes irregulares ataquen  forma violenta  personal  ejército   fronteras inició comentando  edil \nposteriormente agregó   urgente   gobierno   congreso nacional trab

In [10]:
# Comparar las columnas 'cuerpo' y 'cuerpo_limpio'
print(df[['cuerpo', 'cuerpo_limpio']].head())


                                              cuerpo  \
0  Un militar resultó heridos tras un violento en...   
1  La Superintendencia de Electricidad y Combusti...   
2  La mañana de este domingo, la Superintendencia...   
3  La Municipalidad de La Florida entregó 5 mil 6...   
4  El Equipo contra el Crimen Organizado y Homici...   

                                       cuerpo_limpio  
0   militar resultó heridos tras  violento enfren...  
1   superintendencia  electricidad  combustibles ...  
2   mañana   domingo  superintendencia  electrici...  
3   municipalidad   florida entregó 5 mil 600 gif...  
4   equipo   crimen organizado  homicidios ecoh  ...  


"el", "la", "los", "las", así como otras palabras de poca relevancia para el análisis, como conjunciones o preposiciones ("de", "y", "a", etc.). Estas palabras, conocidas como stopwords en procesamiento de lenguaje natural (PLN), a menudo se eliminan porque no aportan un valor significativo en el análisis de texto, especialmente cuando se intenta identificar los términos más relevantes de una noticia.

Por ejemplo:

Texto original:
"Un militar resultó heridos tras un violento enfrentamiento en la frontera."
Texto limpio:
"militar resultó heridos tras violento enfrentamiento frontera"
En este caso, solo se han mantenido las palabras más importantes, que son sustantivos, adjetivos y verbos clave, eliminando los artículos y algunas preposiciones.

¿Por qué se hace esta limpieza?
Simplificación del texto: Facilita el análisis eliminando palabras comunes que no aportan mucho valor semántico.
Reducción de ruido: Las stopwords pueden agregar ruido y afectar el rendimiento de algoritmos de clasificación, modelado o análisis de sentimiento.
Focalización en palabras clave: Se da más importancia a las palabras que aportan significado a la frase.

In [9]:
# Definir las palabras clave para cada categoría y excluir farándula
filtro_economico = r'\becon\w*|finanzas'
filtro_politica = r'política|impuesto|regulaci[oó]n|recaudación|reforma|congreso|senad[oa]|diputad[oa]|gasto fiscal|gasto público|déficit fiscal|deuda pública|presupuesto fiscal|Banco Central|Ministerio de Hacienda'
filtro_incertidumbre = r'incertidumbre|incierto'
filtro_chile = r'\bChile|Chilen[oa]'
filtro_excluir = r'farándula'

# Aplicar el filtro a las columnas 'titulo', 'bajada', y 'cuerpo_limpio'
# Primero aplicamos el filtro económico
filtro_economico_aplicado = df['titulo'].str.contains(filtro_economico, case=False, na=False) | \
                            df['bajada'].str.contains(filtro_economico, case=False, na=False) | \
                            df['cuerpo_limpio'].str.contains(filtro_economico, case=False, na=False)

# Luego aplicamos los filtros para política, incertidumbre y Chile
filtro_politica_aplicado = df['titulo'].str.contains(filtro_politica, case=False, na=False) | \
                           df['bajada'].str.contains(filtro_politica, case=False, na=False) | \
                           df['cuerpo_limpio'].str.contains(filtro_politica, case=False, na=False)

filtro_incertidumbre_aplicado = df['titulo'].str.contains(filtro_incertidumbre, case=False, na=False) | \
                                df['bajada'].str.contains(filtro_incertidumbre, case=False, na=False) | \
                                df['cuerpo_limpio'].str.contains(filtro_incertidumbre, case=False, na=False)

filtro_chile_aplicado = df['titulo'].str.contains(filtro_chile, case=False, na=False) | \
                        df['bajada'].str.contains(filtro_chile, case=False, na=False) | \
                        df['cuerpo_limpio'].str.contains(filtro_chile, case=False, na=False)

# Luego aplicamos el filtro negativo para excluir farándula o similares
filtro_excluir_aplicado = ~(df['titulo'].str.contains(filtro_excluir, case=False, na=False) | \
                            df['bajada'].str.contains(filtro_excluir, case=False, na=False) | \
                            df['cuerpo_limpio'].str.contains(filtro_excluir, case=False, na=False))

# Combinar los filtros: Económico debe ser verdadero Y (Política OR Incertidumbre OR Chile) Y que no sea farándula
filtro_combinado = filtro_economico_aplicado & filtro_excluir_aplicado & \
                   (filtro_politica_aplicado | filtro_incertidumbre_aplicado | filtro_chile_aplicado)

# Aplicar el filtro al DataFrame
df_filtrado = df[filtro_combinado]

# Mostrar las primeras filas del DataFrame filtrado
print(df_filtrado.head(10))


                                                                                                                                           titulo  \
47                                                                Fernanda Villalobos: la chilena que se tituló en la mejor universidad del mundo   
77                                "Nadie merece ser humillado": Maite Orsini llega con el 'Mago' Valdivia y pide justicia contra Daniela Aránguiz   
115                                                                   Boric y elecciones en Venezuela: "No se puede amenazar con baños de sangre"   
116                                                  No habrá indemnización: Suprema rechaza demandas contra SCA Chile por "colusión del confort"   
159                                                     Boric anuncia creación de nueva cárcel de alta seguridad: estará en la RM y con 500 cupos   
174                                             Presentan proyecto de ley que busca prohibir aplicaciones 

In [10]:
df_filtrado.shape

(49680, 8)

In [11]:
# Buscar datos nulos en cada columna
nulos_por_columna = df_filtrado.isnull().sum()

# Mostrar el número de nulos por columna
print(nulos_por_columna)

# También puedes ver si hay nulos en todo el DataFrame
nulos_totales = df_filtrado.isnull().sum().sum()
print(f'Total de valores nulos en el DataFrame: {nulos_totales}')

titulo            8370
bajada            8527
fecha                0
cuerpo               0
fuente               0
url                  0
fecha_scraping       0
cuerpo_limpio        0
dtype: int64
Total de valores nulos en el DataFrame: 16897


In [55]:
#No importan los datos nulos, dado que ocuparemos cuerpo_limpio para hacer el analisis de sentimiento. 

In [13]:
# Guardar el DataFrame como un archivo CSV
ruta = "./noticias_filtradas.csv"
df_filtrado.to_csv(ruta, index=False, encoding='utf-8-sig')  # Utiliza 'utf-8-sig' para manejar caracteres especiales

In [14]:
# Mostrar las primeras filas de la columna 'cuerpo_limpio'
print(df_filtrado['cuerpo_limpio'].head())

47                                                                                                                                                                                                                              maría fernanda villalobos   joven chilena  tras años  esfuerzo logró graduarse  prestigioso instituto tecnológico  massachusetts mit considerado  mejor universidad  mundo según  último ránking qs world university rankings\n historia  fernanda  muestras   gran constancia  esfuerzo    camino hacia  mit   fácil  ello  dudó  sumarse junto   madre   venta  productos   feria libre\n 2019  realizó  primera postulación  encontró  dos impedimentos  postular   ser profesional    10 años  experiencia  contar  amplios recursos económicos\n 2022 volvió  solicitar ingreso   monto necesario   9 años  experiencia laboral tras varios procesos  entrevistas pudo matricularse   convicción  apoyar  desarrollo   mundo mejor así  pasado julio  graduó  programa  alta dirección  sustentabil

In [15]:
# Supongamos que df_filtrado es tu DataFrame
df_filtrado['cuerpo_limpio'] = df_filtrado['cuerpo_limpio'].apply(unidecode)

# Mostrar las primeras filas para verificar
print(df_filtrado['cuerpo_limpio'].head())

47                                                                                                                                                                                                                              maria fernanda villalobos   joven chilena  tras anos  esfuerzo logro graduarse  prestigioso instituto tecnologico  massachusetts mit considerado  mejor universidad  mundo segun  ultimo ranking qs world university rankings\n historia  fernanda  muestras   gran constancia  esfuerzo    camino hacia  mit   facil  ello  dudo  sumarse junto   madre   venta  productos   feria libre\n 2019  realizo  primera postulacion  encontro  dos impedimentos  postular   ser profesional    10 anos  experiencia  contar  amplios recursos economicos\n 2022 volvio  solicitar ingreso   monto necesario   9 anos  experiencia laboral tras varios procesos  entrevistas pudo matricularse   conviccion  apoyar  desarrollo   mundo mejor asi  pasado julio  graduo  programa  alta direccion  sustentabil

C:\Users\chali\AppData\Local\Temp\ipykernel_31460\4056423686.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['cuerpo_limpio'] = df_filtrado['cuerpo_limpio'].apply(unidecode)


In [16]:
df_filtrado.head()

,titulo,bajada,fecha,cuerpo,fuente,url,fecha_scraping,cuerpo_limpio
47,Fernanda Villalobos: la chilena que se tituló en la mejor universidad del mundo,"Trabajó en una feria libre con su madre para costear su educación y estadía en Boston: ""Cada sacrificio valió la pena"", dijo.",2024-08-05,"María Fernanda Villalobos es la joven chilena que, tras años de esfuerzo, logró graduarse del prestigioso Instituto Tecnológico de Massachusetts (MIT), considerado la mejor universidad del mundo según el último ránking QS World University Rankings.\nLa historia de Fernanda es muestras de una gran constancia y esfuerzo, ya que el camino hacia el MIT no fue fácil. Para ello, no dudó en sumarse junto a su madre en la venta de productos en una feria libre.\nEn 2019, cuando realizó su primera postulación, se encontró con dos impedimentos para postular: tenía que ser profesional con más de 10 años de experiencia y contar con amplios recursos económicos.\nEn 2022 volvió a solicitar ingreso, con el monto necesario y con 9 años de experiencia laboral. Tras varios procesos de entrevistas, pudo matricularse, con la convicción de apoyar el desarrollo de un mundo mejor. Así, el pasado julio se graduó del programa de Alta Dirección en Sustentabilidad en el MIT professional education, tema que la apasiona profundamente.\nFernanda logró este éxito enfrentando importantes desafíos: tras estudiar Ingeniería Comercial en la Universidad de Santiago de Chile (Usach), trabajó en la industria farmacéutica nacional (la start-up Fracción), y, para terminar de costear su educación y estadía en Boston, trabajó en una feria libre local junto a su madre, profesora de profesión, persiguiendo sus sueños con una determinación inquebrantable.\nA pesar de las largas horas de trabajo y los desafíos financieros, nunca perdió su enfoque ni su pasión.\n""Me gustan los desafíos porque provengo de una familia de mujeres fuertes y decididas y quiero que mi hija sea igual. Estudiar en el MIT no es un azar del destino, sino un objetivo que me planteé y seguí sin descanso, porque eso era lo que yo quería lograr y, finalmente, no sólo me titulé siendo la más joven del programa. Quería aprender de los mejores porque mi meta final es transformarme en una líder del tema en el país"", comentó.\n""Cada sacrificio valió la pena, y estoy agradecida por todas las oportunidades y experiencias que viví en el MIT. Ahora, quiero aplicar mis conocimientos para ayudar a mi comunidad y contribuir al desarrollo de la sostenibilidad en Chile"", afirmó.\nA mediados de julio de este año, Fernanda viajó a recibir su título. Pero antes debió realizar un último trabajo en equipo para la titulación. Para ello decidieron trabajar el concepto 'reducción del plástico de un solo uso'.\n""El proyecto no sólo fue aprobado, sino que, además, nos dieron el premio ‘Fire Hydrant Award’ que simboliza la resiliencia, consistencia y determinación que se necesita en el MIT"", describió.",24horas,https://www.24horas.cl/actualidad/nacional/fernanda-villalobos-chilena-titulo-mejor-universidad,2024-08-12 09:49:58.812149,maria fernanda villalobos joven chilena tras anos esfuerzo logro graduarse prestigioso instituto tecnologico massachusetts mit considerado mejor universidad mundo segun ultimo ranking qs world university rankings\n historia fernanda muestras gran constancia esfuerzo camino hacia mit facil ello dudo sumarse junto madre venta productos feria libre\n 2019 realizo primera postulacion encontro dos impedimentos postular ser profesional 10 anos experiencia contar amplios recursos economicos\n 2022 volvio solicitar ingreso monto necesario 9 anos experiencia laboral tras varios procesos entrevistas pudo matricularse conviccion apoyar desarrollo mundo mejor asi pasado julio graduo programa alta direccion sustentabilidad mit professional education tema apasiona profundamente\nfernanda logro exito enfrentando importantes desafios tras estudiar ingenieria comercial universidad santiago chile usach trabajo industria farma